In [3]:
import requests
import pandas as pd
import json
from datetime import datetime, timedelta

In [4]:
def fetch_schedule_for_week(date):
    """Fetch the NHL Schedule for a given date"""
    url = f"https://api-web.nhle.com/v1/schedule/{date}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        game_weeks = data.get("gameWeek", [])

        schedule_data = []

        for week in game_weeks:
            date = week.get("date")
            games = week.get("games", [])

            for game in games:
                schedule_data.append({
                    "game_id": game.get("id"),
                    "date": date,
                    "season": game.get("season"),
                    "game_type": game.get("gameType"),
                    "venue": game.get("venue", {}).get("default"),
                    "neutral_site": game.get("neutralSite"),
                    "start_time": game.get("startTimeUTC"),
                    "home_team": game.get("homeTeam", {}).get("placeName", {}).get("default"),
                    "home_team_score": game.get("homeTeam", {}).get("score"),
                    "away_team": game.get("awayTeam", {}).get("placeName", {}).get("default"),
                    "away_team_score": game.get("awayTeam", {}).get("score"),
                    "game_state": game.get("gameState"),
                    "game_center_link": game.get("gameCenterLink")
                })
        


        df = pd.DataFrame(schedule_data)
        return df

    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch schedule data: {e}")
        return pd.DataFrame()

In [6]:
date = "2024-10-18"
nhlSchedule2024 = fetch_schedule_for_week(date)

nhlSchedule2024 = nhlSchedule2024[nhlSchedule2024['game_state'] == 'OFF']
nhlSchedule2024

,game_id,date,season,game_type,venue,neutral_site,start_time,home_team,home_team_score,away_team,away_team_score,game_state,game_center_link
0,2024020072,2024-10-18,20242025,2,PPG Paints Arena,False,2024-10-18T23:00:00Z,Pittsburgh,1,Carolina,4,OFF,/gamecenter/car-vs-pit/2024/10/18/2024020072
1,2024020073,2024-10-18,20242025,2,Canada Life Centre,False,2024-10-19T00:00:00Z,Winnipeg,8,San Jose,3,OFF,/gamecenter/sjs-vs-wpg/2024/10/18/2024020073
2,2024020074,2024-10-18,20242025,2,Ball Arena,False,2024-10-19T01:00:00Z,Colorado,4,Anaheim,3,OFF,/gamecenter/ana-vs-col/2024/10/18/2024020074
3,2024020075,2024-10-19,20242025,2,Canadian Tire Centre,False,2024-10-19T17:00:00Z,Ottawa,5,Tampa Bay,4,OFF,/gamecenter/tbl-vs-ott/2024/10/19/2024020075
4,2024020076,2024-10-19,20242025,2,Bridgestone Arena,False,2024-10-19T18:00:00Z,Nashville,2,Detroit,5,OFF,/gamecenter/det-vs-nsh/2024/10/19/2024020076
5,2024020077,2024-10-19,20242025,2,American Airlines Center,False,2024-10-19T20:00:00Z,Dallas,4,Edmonton,1,OFF,/gamecenter/edm-vs-dal/2024/10/19/2024020077
6,2024020078,2024-10-19,20242025,2,Amerant Bank Arena,False,2024-10-19T22:00:00Z,Florida,4,Vegas,3,OFF,/gamecenter/vgk-vs-fla/2024/10/19/2024020078
7,2024020079,2024-10-19,20242025,2,Scotiabank Arena,False,2024-10-19T23:00:00Z,Toronto,1,New York,4,OFF,/gamecenter/nyr-vs-tor/2024/10/19/2024020079
8,2024020080,2024-10-19,20242025,2,Prudential Center,False,2024-10-19T23:00:00Z,New Jersey,5,Washington,6,OFF,/gamecenter/wsh-vs-njd/2024/10/19/2024020080
9,2024020081,2024-10-19,20242025,2,UBS Arena,False,2024-10-19T23:00:00Z,New York,4,Montréal,3,OFF,/gamecenter/mtl-vs-nyi/2024/10/19/2024020081


In [11]:
def fetch_schedule_for_year(start_year):
    """ Fetches the schedule for the whole year (i.e. fetch_schedule_for_year(2024) = 2024-25 NHL Schedule"""

    start_date = f"{start_year}-10-01"
    end_date = f"{start_year + 1}-07-15"

    current_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")

    entire_schedule_data = []

    while current_date <= end_date:

        date_str = current_date.strftime("%Y-%m-%d")

        week_schedule = fetch_schedule_for_week(date_str)

        if not week_schedule.empty:
            entire_schedule_data.append(week_schedule)

            last_date_str = week_schedule['date'].iloc[-1]
            last_date = datetime.strptime(last_date_str, "%Y-%m-%d")
            current_date = last_date + timedelta(days=1)
        else:
            break

    if entire_schedule_data:
        entire_schedule_df = pd.concat(entire_schedule_data, ignore_index=True)
        return entire_schedule_df
    else:
        print("No schedule data found for the specified year.")
        return pd.DataFrame()    

In [33]:
nhl_schedule_2024 = fetch_schedule_for_year(2023)

In [34]:
nhl_schedule_2024 = nhl_schedule_2024[nhl_schedule_2024['game_id'] < 2023021315]
nhl_schedule_2024

,game_id,date,season,game_type,venue,neutral_site,start_time,home_team,home_team_score,away_team,away_team_score,game_state,game_center_link
0,2023010065,2023-10-01,20232024,1,Centre 200,True,2023-10-01T20:00:00Z,Ottawa,4,Florida,2,FINAL,/gamecenter/fla-vs-ott/2023/10/01/2023010065
1,2023010066,2023-10-01,20232024,1,Acrisure Arena,True,2023-10-01T22:00:00Z,Arizona,4,Anaheim,5,FINAL,/gamecenter/ana-vs-ari/2023/10/01/2023010066
2,2023010067,2023-10-01,20232024,1,Ball Arena,False,2023-10-01T23:00:00Z,Colorado,4,Dallas,3,FINAL,/gamecenter/dal-vs-col/2023/10/01/2023010067
3,2023010068,2023-10-01,20232024,1,Little Caesars Arena,False,2023-10-01T23:00:00Z,Detroit,6,Chicago,1,FINAL,/gamecenter/chi-vs-det/2023/10/01/2023010068
4,2023010069,2023-10-02,20232024,1,Scotiabank Centre,True,2023-10-02T22:00:00Z,Pittsburgh,0,Ottawa,3,FINAL,/gamecenter/ott-vs-pit/2023/10/02/2023010069
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358,2023021308,2024-04-18,20232024,2,Canada Life Centre,False,2024-04-19T00:00:00Z,Winnipeg,4,Vancouver,2,OFF,/gamecenter/van-vs-wpg/2024/04/18/2023021308
1359,2023021309,2024-04-18,20232024,2,Scotiabank Saddledome,False,2024-04-19T01:00:00Z,Calgary,5,San Jose,1,OFF,/gamecenter/sjs-vs-cgy/2024/04/18/2023021309
1360,2023021310,2024-04-18,20232024,2,Ball Arena,False,2024-04-19T01:30:00Z,Colorado,5,Edmonton,1,OFF,/gamecenter/edm-vs-col/2024/04/18/2023021310
1361,2023021311,2024-04-18,20232024,2,T-Mobile Arena,False,2024-04-19T02:00:00Z,Vegas,1,Anaheim,4,OFF,/gamecenter/ana-vs-vgk/2024/04/18/2023021311
